<a href="https://colab.research.google.com/github/sameerdewan/DecentralizedHousingMarket/blob/master/Hackathon_ATL_Guardian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests opencv-python opencv-python-headless numpy pytz supervision inference

In [ ]:
import os
import signal
import sys
import time
from datetime import datetime

import cv2
import numpy as np
import requests
import pytz
import supervision as sv
from inference import get_model
import matplotlib.pyplot as plt

# Configuration
IMAGE_URL = 'https://webcams.nyctmc.org/api/cameras/04e09ed5-2d97-4e29-8438-b87748850dbb/image'  # Replace with your image URL
FRAME_RATE = 1  # frames per second
VIDEO_SIZE = (640, 480)  # Set your desired video size
OUTPUT_FILE = "annotated_timelapse.mp4"

# Initialize model
model = get_model(model_id="yolov8n-640")

def process_image(image_url):
    # Perform inference
    results = model.infer(
        image=image_url,
        confidence=0.1,
        iou_threshold=0.1
    )

    print(results)

    detections = results[0].predictions

    # Fetch the image to annotate
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        response.raw.decode_content = True
        image = np.asarray(bytearray(response.raw.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    else:
        print(f"Failed to fetch image for annotation. Status code: {response.status_code}")
        return None

    # Annotate frame
    for det in detections:
        x1 = int(det.x - det.width / 2)
        y1 = int(det.y - det.height / 2)
        x2 = int(det.x + det.width / 2)
        y2 = int(det.y + det.height / 2)
        label = f"{det.class_name}: {det.confidence:.2f}"
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image

def setup_video_writer(filename, size, fps):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(filename, fourcc, fps, size)
    return writer

def setup_signal_handler(writer):
    def signal_handler(sig, frame):
        writer.release()
        sys.exit(0)

    signal.signal(signal.SIGINT, signal_handler)

def display_image(image):
    # Convert the image from BGR to RGB format
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')  # Turn off axis labels
    plt.show()

def main():
    writer = setup_video_writer(OUTPUT_FILE, VIDEO_SIZE, FRAME_RATE)
    setup_signal_handler(writer)

    try:
        while True:
            # Process the image URL directly
            annotated_image = process_image(IMAGE_URL)

            if annotated_image is not None:
                # Resize the image to match video size
                annotated_image = cv2.resize(annotated_image, VIDEO_SIZE)

                # Write the frame to the video file
                writer.write(annotated_image)

                # Display the image
                display_image(annotated_image)

            # Wait for the next frame
            time.sleep(1 / FRAME_RATE)
    finally:
        writer.release()

if __name__ == "__main__":
    main()
